In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
import tls_client

In [2]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
requests = tls_client.Session(
    client_identifier="chrome112",
)
response1 = requests.get(dog_props_url)

data = response1.json()

In [3]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [4]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
2259,b927f037-d3ea-403c-8d5b-f982422eee31,False,None,[{'id': 'c3db4c46-4923-4069-8551-aa97ef47c4af'...,c225e553-6dbb-4388-b2e0-0ee55ef89da1,25114110,76.5,active,c225e553-6dbb-4388-b2e0-0ee55ef89da1,dc37680c-346c-4c47-94b6-28d832afe988,f727e378-7fda-4d64-8e8a-2d159a44123c,Fantasy Points in Games 1+2,high_score,d8f7b19e-ddaa-4159-9310-65a1dddbfd34,fantasy_points_in_games_1_2,None,None,Dota: Lou Fantasy Points in Games 1+2 O/U
356,3e5a5e36-a340-491f-92ff-44f807165379,False,None,[{'id': '193133d8-1bb5-48af-ae8f-a6f00e91929e'...,f6ab9e32-6b0a-4fa1-8e9f-ccef7b779fa9,2114001,272.5,active,f6ab9e32-6b0a-4fa1-8e9f-ccef7b779fa9,088f02f5-cf21-4f6a-9d88-1f67509068f3,4ddc12d3-06e4-4d83-8d1d-fb54bd91dfa5,Passing Yards,high_score,55d1a168-e9bf-4a40-8440-fb8a7a07713a,passing_yds,None,None,Patrick Mahomes Passing Yards O/U


In [5]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [6]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [7]:
player_df

,full_name,sport_id,team_id
0,Baker Mayfield,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
1,Rachaad White,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
2,Mike Evans,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
3,Chris Godwin,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
4,Cade Otton,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
...,...,...,...
850,N. Anderson + D. Stoops,CFBCOMBOS,059969e5-8906-4018-b150-fc4e94a6ea06
851,X. Legette + T. Knox,CFBCOMBOS,e9a34d04-a232-4e39-bf56-9f12e5810a22
852,E. Stewart + A. Smith,CFBCOMBOS,efd4c1b7-ddf1-4d3e-a5f9-c66c02c1be29
853,E. Wilson + A. Boardingham,CFBCOMBOS,0ca88a9f-5ce8-4785-8d78-f3947791cba9


In [8]:
player_df.sport_id.unique()

array(['NFL', 'NBA', 'MLB', 'NHL', 'CFB', 'FIFA', 'ESPORTS', 'RACING',
       'SOCCER', 'NFLCOMBO', 'NBACOMBOS', 'MLBCOMBOS', 'CFBCOMBOS'],
      dtype=object)

In [9]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [10]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
69,56945.0,da6ba87f-d141-4f1c-952f-931e1e55480f,0.0,c6d8a870-3875-41d5-9f2d-45ed155ec525,0.0,Thu 08:00pm,0.0,2023-10-27T00:00:00Z,regular,ESPORTS,scheduled,CS2: Mythic vs. Eros,Game,2023.0
70,57045.0,ab795a2e-b243-4eda-be0f-024477c452f1,0.0,5d691864-302e-44c8-b624-e7a2031d2616,0.0,Fri 09:00pm,0.0,2023-10-28T01:00:00Z,regular,SOCCER,scheduled,Pumas vs Necaxa,Game,2023.0
71,57046.0,647ac9f8-16c3-4bf3-8d5d-42279852b3c6,0.0,d0b6f8db-7035-4076-b436-e98853a600a7,0.0,Fri 11:00pm,0.0,2023-10-28T03:00:00Z,regular,SOCCER,scheduled,Queretaro vs Mazatlan,Game,2023.0


In [11]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [12]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [13]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [14]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,39413.0,1a20f1da-c502-5224-9c4a-bc363174cd21,0.0,0719b253-43db-532e-8b5c-6c12c6c3f951,0.0,Thu 08:15pm,0.0,2023-10-26,regular,NFL,scheduled,TB @ BUF,Game,2023.0
1,39422.0,d150534e-6a05-587b-b9e3-50ef86602e20,0.0,26e67e06-664e-50a6-ad7b-c102705fde8b,0.0,Sun 01:00pm,0.0,2023-10-29,regular,NFL,scheduled,LAR @ DAL,Game,2023.0
2,39416.0,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6,0.0,ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8,0.0,Sun 01:00pm,0.0,2023-10-29,regular,NFL,scheduled,JAX @ PIT,Game,2023.0
3,39414.0,de7219e5-92f1-5989-804a-68479055ba42,0.0,a6d8dc19-daaf-5798-a8f2-df7f9fc9eecd,0.0,Sun 01:00pm,0.0,2023-10-29,regular,NFL,scheduled,PHI @ WAS,Game,2023.0
4,39419.0,ef876de9-81ac-5e60-af6e-0ca1700f3a51,0.0,9153e0a9-da83-5246-ba43-417537f1bcce,0.0,Sun 01:00pm,0.0,2023-10-29,regular,NFL,scheduled,NE @ MIA,Game,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,57052.0,7e1c9894-9a26-4697-b5b8-be923e9ba520,0.0,c8b97fd2-4ee9-4741-a1c8-d5a4be09cf2a,0.0,Thu 05:00pm,0.0,2023-10-26,regular,ESPORTS,scheduled,CS2: Solid vs. BESTIA,Game,2023.0
68,57053.0,1f9cfdc5-2e55-4a48-a1b6-77e9fad54ea7,0.0,35b4a870-f480-4347-bf9d-d6b4801e79fa,0.0,Thu 08:00pm,0.0,2023-10-26,regular,ESPORTS,scheduled,CS2: Meta vs. Filhos de D10S,Game,2023.0
69,56945.0,da6ba87f-d141-4f1c-952f-931e1e55480f,0.0,c6d8a870-3875-41d5-9f2d-45ed155ec525,0.0,Thu 08:00pm,0.0,2023-10-26,regular,ESPORTS,scheduled,CS2: Mythic vs. Eros,Game,2023.0
70,57045.0,ab795a2e-b243-4eda-be0f-024477c452f1,0.0,5d691864-302e-44c8-b624-e7a2031d2616,0.0,Fri 09:00pm,0.0,2023-10-27,regular,SOCCER,scheduled,Pumas vs Necaxa,Game,2023.0


In [15]:
id2tm

{'0719b253-43db-532e-8b5c-6c12c6c3f951': 'BUF',
 '26e67e06-664e-50a6-ad7b-c102705fde8b': 'DAL',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 'a6d8dc19-daaf-5798-a8f2-df7f9fc9eecd': 'WAS',
 '9153e0a9-da83-5246-ba43-417537f1bcce': 'MIA',
 'f11f1cf1-9933-5203-8181-95020ee64399': 'IND',
 'd40a4380-49a6-5b5f-ab7d-f5393787ed12': 'NYG',
 'a2fadaac-562f-5a7e-bd63-6d88d90c1ac4': 'GB',
 'f96aa8db-21c2-5b86-b49d-7e64b4eda61d': 'TEN',
 '17f3bc4a-e2d6-5dc5-a554-00549ff0139f': 'CAR',
 '31631011-5902-52f6-ba01-c4c8d8eb3fd9': 'SEA',
 '7161e62b-de20-56e2-a300-0dc23637faaa': 'SF',
 '7b3b21be-a209-5dee-a3f7-3fae61f52a1e': 'DEN',
 '8459772c-695a-5890-afa1-7ec38da17201': 'ARI',
 'c7b497d4-18b6-522b-abaa-e5c3d24bc021': 'LAC',
 'a8980eb6-327f-5bc5-abf9-de1e944b566d': 'DET',
 'e142b705-dd0a-4803-8cde-a35788a65341': 'MIL',
 '97616c91-0509-42f8-99f3-04f3856a6377': 'LAL',
 '56a5d8c8-eb72-47db-9e93-5b24d8f5443a': 'MEM',
 '8f5852b6-c8dc-4cce-980a-eab09b28919a': 'BOS',
 'f85132d6-ce4f-461f-ab0d-97810d017c94': '

In [16]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [17]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Baker Mayfield', 'Rachaad White', 'Mike Evans', 'Chris Godwin',
       'Cade Otton', 'Chase McLaughlin', 'Trey Palmer', 'Deven Thompkins',
       'Ryan Neal', 'Devin White', 'Jamel Dean', 'Lavonte David',
       'Josh Allen', 'James Cook', 'Latavius Murray', 'Stefon Diggs',
       'Gabe Davis', 'Dalton Kincaid', 'Khalil Shakir', 'Tyler Bass',
       'Taron Johnson', 'Micah Hyde', 'Terrel Bernard', 'Jordan Poyer',
       'Matthew Stafford', 'Cooper Kupp', 'Puka Nacua', 'Tutu Atwell',
       'Tyler Higbee', 'Dak Prescott', 'Tony Pollard', 'CeeDee Lamb',
       'Brandin Cooks', 'Michael Gallup', 'Jake Ferguson',
       'Brandon Aubrey', 'Trevor Lawrence', 'Travis Etienne Jr.',
       'Calvin Ridley', 'Christian Kirk', 'Evan Engram',
       'Brandon McManus', 'Rayshawn Jenkins', 'Devin Lloyd',
       'Kenny Pickett', 'Najee Harris', 'Jaylen Warren', 'Chris Boswell',
       'Jalen Hurts', "D'Andre Swift", 'A.J. Brown', 'DeVonta Smith',
       'Dallas Goedert', 'Jake Elliott', 'Sam H

In [18]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NFL', 'NFL', 'NFL', 'NFL', 'NFL']

In [19]:
guys[1000:1007]

['Evan Carter',
 'Jonah Heim',
 'Leody Taveras',
 'Evan Carter',
 'Joey Daccord',
 'Joey Daccord',
 'Jared McCann']

In [20]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['1a20f1da-c502-5224-9c4a-bc363174cd21',
 '1a20f1da-c502-5224-9c4a-bc363174cd21',
 '1a20f1da-c502-5224-9c4a-bc363174cd21',
 '1a20f1da-c502-5224-9c4a-bc363174cd21',
 '1a20f1da-c502-5224-9c4a-bc363174cd21']

In [21]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,230.5,passing_yds,Baker Mayfield Passing Yards O/U,Baker Mayfield,NFL
1,10.5,rushing_yds,Baker Mayfield Rushing Yards O/U,Baker Mayfield,NFL
2,2.5,rushing_att,Baker Mayfield Rushing Attempts O/U,Baker Mayfield,NFL
3,34.5,passing_att,Baker Mayfield Passing Attempts O/U,Baker Mayfield,NFL
4,21.5,passing_comps,Baker Mayfield Completions O/U,Baker Mayfield,NFL
...,...,...,...,...,...
2451,118.5,receiving_yds,N. Anderson + D. Stoops Receiving Yards O/U,N. Anderson + D. Stoops,other
2452,91.5,receiving_yds,X. Legette + T. Knox Receiving Yards O/U,X. Legette + T. Knox,other
2453,124.5,receiving_yds,E. Stewart + A. Smith Receiving Yards O/U,E. Stewart + A. Smith,other
2454,64.5,receiving_yds,E. Wilson + A. Boardingham Receiving Yards O/U,E. Wilson + A. Boardingham,other


In [22]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,230.5,passing_yds,Baker Mayfield Passing Yards O/U,Baker Mayfield,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
1,10.5,rushing_yds,Baker Mayfield Rushing Yards O/U,Baker Mayfield,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
2,2.5,rushing_att,Baker Mayfield Rushing Attempts O/U,Baker Mayfield,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
3,34.5,passing_att,Baker Mayfield Passing Attempts O/U,Baker Mayfield,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
4,21.5,passing_comps,Baker Mayfield Completions O/U,Baker Mayfield,NFL,1a20f1da-c502-5224-9c4a-bc363174cd21
...,...,...,...,...,...,...
2451,118.5,receiving_yds,N. Anderson + D. Stoops Receiving Yards O/U,N. Anderson + D. Stoops,other,059969e5-8906-4018-b150-fc4e94a6ea06
2452,91.5,receiving_yds,X. Legette + T. Knox Receiving Yards O/U,X. Legette + T. Knox,other,e9a34d04-a232-4e39-bf56-9f12e5810a22
2453,124.5,receiving_yds,E. Stewart + A. Smith Receiving Yards O/U,E. Stewart + A. Smith,other,efd4c1b7-ddf1-4d3e-a5f9-c66c02c1be29
2454,64.5,receiving_yds,E. Wilson + A. Boardingham Receiving Yards O/U,E. Wilson + A. Boardingham,other,0ca88a9f-5ce8-4785-8d78-f3947791cba9


In [23]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [24]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,230.5,passing_yds,Baker Mayfield Passing Yards O/U,Baker Mayfield,NFL,TB,2023-10-26
1,10.5,rushing_yds,Baker Mayfield Rushing Yards O/U,Baker Mayfield,NFL,TB,2023-10-26
2,2.5,rushing_att,Baker Mayfield Rushing Attempts O/U,Baker Mayfield,NFL,TB,2023-10-26
3,34.5,passing_att,Baker Mayfield Passing Attempts O/U,Baker Mayfield,NFL,TB,2023-10-26
4,21.5,passing_comps,Baker Mayfield Completions O/U,Baker Mayfield,NFL,TB,2023-10-26
...,...,...,...,...,...,...,...
2451,118.5,receiving_yds,N. Anderson + D. Stoops Receiving Yards O/U,N. Anderson + D. Stoops,other,059969e5-8906-4018-b150-fc4e94a6ea06,NaN
2452,91.5,receiving_yds,X. Legette + T. Knox Receiving Yards O/U,X. Legette + T. Knox,other,e9a34d04-a232-4e39-bf56-9f12e5810a22,NaN
2453,124.5,receiving_yds,E. Stewart + A. Smith Receiving Yards O/U,E. Stewart + A. Smith,other,efd4c1b7-ddf1-4d3e-a5f9-c66c02c1be29,NaN
2454,64.5,receiving_yds,E. Wilson + A. Boardingham Receiving Yards O/U,E. Wilson + A. Boardingham,other,0ca88a9f-5ce8-4785-8d78-f3947791cba9,NaN


In [25]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [26]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,230.5,passing_yds,Baker Mayfield,NFL,TB,2023-10-26
1,10.5,rushing_yds,Baker Mayfield,NFL,TB,2023-10-26
2,2.5,rushing_att,Baker Mayfield,NFL,TB,2023-10-26
3,34.5,passing_att,Baker Mayfield,NFL,TB,2023-10-26
4,21.5,passing_comps,Baker Mayfield,NFL,TB,2023-10-26
...,...,...,...,...,...,...
2451,118.5,receiving_yds,N. Anderson + D. Stoops,other,059969e5-8906-4018-b150-fc4e94a6ea06,NaN
2452,91.5,receiving_yds,X. Legette + T. Knox,other,e9a34d04-a232-4e39-bf56-9f12e5810a22,NaN
2453,124.5,receiving_yds,E. Stewart + A. Smith,other,efd4c1b7-ddf1-4d3e-a5f9-c66c02c1be29,NaN
2454,64.5,receiving_yds,E. Wilson + A. Boardingham,other,0ca88a9f-5ce8-4785-8d78-f3947791cba9,NaN


In [27]:
df = df.reset_index(drop=True)

In [28]:
df.Stat.unique()

array(['passing_yds', 'rushing_yds', 'rushing_att', 'passing_att',
       'passing_comps', 'passing_first_downs', 'passing_comp_pct',
       'passing_times_sacked', 'passing_and_rushing_yds',
       'fantasy_points', 'receiving_yds', 'rush_rec_yds', 'receiving_rec',
       'first_downs', 'receiving_yac', 'receiving_tgts',
       'extra_points_made', 'kicking_points', 'tackles',
       'tackles_and_assists', 'assists', 'passing_ints',
       'field_goals_made', 'passing_tds', 'rush_rec_tds', 'sacks',
       'double_doubles', 'points', 'pts_rebs_asts', 'rebounds',
       'rebs_asts', 'pts_asts', 'pts_rebs', 'turnovers',
       'three_points_att', 'field_goals_att', 'free_throws_made',
       'blks_stls', 'blocks', 'three_points_made', 'personal_fouls',
       'steals', 'strikeouts', 'runs_allowed', 'hits_runs_rbis',
       'singles', 'total_bases', 'pitch_outs', 'hits_allowed', 'runs',
       'saves', 'shots', 'goals_against', 'goals', 'power_play_points',
       'shots_attempted', 'goal

In [29]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [30]:
df = df.replace(syntax)

In [31]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
1696,13.0,kills_in_maps_1_2,Canna+Canyon,ESPORTS,Dplus KIA,2023-10-27
1697,20.0,kills_in_maps_1_2,Deft+ShowMaker,ESPORTS,Dplus KIA,2023-10-27
1698,29.3,fantasy_points_on_map_1,Canna,ESPORTS,Dplus KIA,2023-10-27
1699,55.9,fantasy_points_on_maps_1_2,Canna,ESPORTS,Dplus KIA,2023-10-27
1700,6.5,kills_in_maps_1_2,Canna,ESPORTS,Dplus KIA,2023-10-27
...,...,...,...,...,...,...
2354,13.0,headshots_on_maps_1_2,fl0m,ESPORTS,Mythic,2023-10-26
2355,15.5,headshots_on_maps_1_2,freakazoid,ESPORTS,Mythic,2023-10-26
2356,15.5,headshots_on_maps_1_2,trucklover86,ESPORTS,Mythic,2023-10-26
2357,16.0,headshots_on_maps_1_2,hate,ESPORTS,Mythic,2023-10-26


In [32]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [33]:
today = dt.date.today()

In [34]:
str(today)

'2023-10-26'

In [35]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [36]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
1696,13.0,kills_in_maps_1_2,Canna+Canyon,ESPORTS,Dplus KIA,2023-10-27
1697,20.0,kills_in_maps_1_2,Deft+ShowMaker,ESPORTS,Dplus KIA,2023-10-27
1698,29.3,fantasy_points_on_map_1,Canna,ESPORTS,Dplus KIA,2023-10-27
1699,55.9,fantasy_points_on_maps_1_2,Canna,ESPORTS,Dplus KIA,2023-10-27
1700,6.5,kills_in_maps_1_2,Canna,ESPORTS,Dplus KIA,2023-10-27
...,...,...,...,...,...,...
2354,13.0,headshots_on_maps_1_2,fl0m,ESPORTS,Mythic,2023-10-26
2355,15.5,headshots_on_maps_1_2,freakazoid,ESPORTS,Mythic,2023-10-26
2356,15.5,headshots_on_maps_1_2,trucklover86,ESPORTS,Mythic,2023-10-26
2357,16.0,headshots_on_maps_1_2,hate,ESPORTS,Mythic,2023-10-26
